In [2]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# ConversationKGMemory:
# - 지식 그래프(Knowledge Graph) 구조를 활용한 메모리
# - 대화 중 엔티티(주요 대상, 인물, 사물)을 추출하고 그들 사이의 관계를 저장함
memory = ConversationKGMemory(
    llm = llm,
    return_messages=True,
)

def add_message(input, output):
    # 대화 저장 시 LLM이 엔티티와 관계를 분석해서 그래프에 기록
    memory.save_context({"input": input}, {"outout": output})


add_message("Hi I'm Yujin, I live in South Korea", "Wow that is so cool!")


In [ ]:
# 특정 엔티티와 관련된 지식만 추출
# 질문(input) 속의 엔티티 'Yujin'과 관련된 정보만 불러옴
memory.load_memory_variables({"input": "who is Yujin"})

{'history': [SystemMessage(content='On Yujin: Yujin lives in South Korea.')]}

In [ ]:
add_message("Yujin likes Dubai chewy cookie", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "what does Yujin like"})

{'history': [SystemMessage(content='On Yujin: Yujin lives in South Korea. Yujin likes Dubai chewy cookie.')]}